In [1]:
import os
import random
import re
import frontmatter

In [2]:
TIDDLER_FOLDER = 'tiddlers'
OUT_FOLDER = 'output'

In [3]:
tiddlers = [t for t in os.listdir(TIDDLER_FOLDER) if '.tid' in t]

In [4]:
len(tiddlers)

702

## Load in the tiddlers

In [5]:
tids_plain = {}

for t in tiddlers:
    tid_path = os.path.join(TIDDLER_FOLDER, t)
    name = os.path.splitext(t)[0]
    with open(tid_path, 'r') as infile:
        tids_plain[name] = infile.read()

print(len(tids_plain))

702


## Turn the plain text into objects

In [6]:
errors = []
tid_objs = {}

for name, text in tids_plain.items():
    
    # format frontmatter
    
    # hack to define end of frontmatter
    text = text.replace('type: text/vnd.tiddlywiki', '---')
    
    # get rid of square brackets in the frontmater
    splitty = text.split('---')
    front = splitty[0]
    front = front.replace('[', '')
    front = front.replace(']', '')
    splitty[0] = front
    text = '---'.join(splitty)
    
    # define start of frontmatter
    text = '---\n' + text
    
    # get rid of hashtag in frontmatter value
    text = text.replace('#00A560', '00A560')
    
    # list any errors so they can be dealt with manually
    try:
        data = frontmatter.loads(text)
        tid_objs[name] = data
    except:
        errors.append(name)
    
print(errors)

['3dArtist', 'QuinoaChickpeaBeanSalad', 'KhanLearningMythArticle', 'GlobalWarmingClimateScienceCoursera', 'DuravitShowerTrayEstimate']


In [7]:
# find all metadata fields in all tiddlers

all_fields = []

for name, ob in tid_objs.items():
    for key in ob.keys():
        all_fields.append(key)

all_fields = set(all_fields)
print(' '.join(all_fields))

subtiddler-code start-hour job-number production-path image-url project timestamp birthday in-state-tuition genre recommended-by caption software evaluation school description stop medium location interests client-code author record-type twitter icon father org-type container date participants pp publication dynalist-link draft.title skill-level locale title domain image-qty team-members email owned modified creator start list instagram preferred-image-dimensions modifier profession status stop-hour client family stop-minute color source tags draft.of profile rating app-deadline time created production-folder-name phone img-url job-title organization animation-qty


In [8]:
# fields to move to the body of the document where they can become links

fields_to_move = [
    'client',
    'software',
    'school',
    'organization',
    'container',
    'participants',
    'description',
    'family',
    'author',
    'profession',
    'publication',
    'project',
    'father',
    'source',
]

In [9]:
# show the values of the fields I'm interested in moving

fields_of_interest_values = []

for name, ob in tid_objs.items():
    for field in ob.keys():
        if field in fields_to_move:
            fields_of_interest_values.append(ob[field])

fields_of_interest_values = set(fields_of_interest_values)

In [10]:
for name, ob in tid_objs.items():

    fields_string = ''
        
    for field in fields_to_move:
        value = ob.get(field)
        if value:
            fields_string += f'\n{field}: {value}'
    
    ob.content += fields_string

## Fix tags TO-DO: fix empty brackets appending to files

There were only a few tags I wanted to keep as tags, the rest I was content to turn into links appended at the bottom of the note.

In [11]:
all_tags = []

for name, ob in tid_objs.items():
    tags_string = ob.get('tags')
    if not tags_string:
        continue
    tags = tags_string.split(' ')
    for t in tags:
        all_tags.append(t)

all_tags = set(all_tags)
print(' '.join(all_tags))

Shaders BestPractice School Place RenderingSuccess Seed NeedsExcision Meta OpenSource Index Simulation Language Productivity NeedsAttention Reference Hardware Power Compositing Achievement NeedsImage Aesthetics Mayo Survival Git Windows Python Knowledge Concept Factory SceneSetup Software MyStory Dashboard Flora Homebrew Artifact Bug Travel Security Solved Scripting Client Nodes Innovation Dev Metaphysics Modularity Record Template Industry Food Animation Tip TiddlyWiki HealthAndFitness Source Project Scholarship Stub Writing Definition Photoshop Publication MayoSite PAO Chess GravCMS Blender Journal Academia Experience Admissions Problem Math Design Physics Economics FrontEnd NodeJs Versioning Art Psychology Workflow Modeling3D SubstanceDesigner Chapter Music Guide ComputerScience


In [12]:
tags_to_keep = [
    'pao',
    'journal',
    'guide',
    'bug',
    'seed',
    'place',
    'chapter',
    'meta',
    'tip',
    'source',
    'guide',
    'concept',
    'project',
    'artifact',
    'index',
    'template',
    'software',
    'dashboard'
]

In [13]:
for name, ob in tid_objs.items():
    
    # get the tags from the metadata
    tags_string = ob.get('tags')
    if not tags_string:
        continue
        
    tags_string = tags_string.replace('Record', '')
    tags_string = tags_string.replace('Solved', '')
    tags_string = tags_string.replace('Stub', '')
        
    # keep lowercase versions of some tags
    tags = tags_string.split(' ')
    to_keep = [t for t in tags if t.lower() in tags_to_keep]
    ob['tags'] = ' '.join(to_keep).lower()
    
    # put the rest of the tags at the bottom as links
    topics = [t for t in tags if not t.lower() in tags_to_keep]
    topics = [f'[[{x}]]' for x in topics]
    ob.content += f'\n\n{" ".join(topics)}'

## Fix Markdown syntax

In [14]:
for name, ob in tid_objs.items():
    
    content = ob.content
    
    if not content:
        continue
    
    # fix image attachments
    content = content.replace('[img', '![')
    
    # fix bolding
    content = content.replace("''", "**")
    
    ob.content = content

## Fix Aliases

TiddlyWiki aliases are backward compared to Obsidian flavored aliases. In Obsidian, the title comes first and then the alias.

In [15]:
# Regex to match alias links
alias_ex = re.compile(r'\[\[.*?\|.*?\]\]')

In [16]:
# Make a function to swap the first part of an alias with the second

test_alias = r"[[i'm aliased text|Note Title]]"

def swap_alias(match):
    body = match.strip('[]')
    alias, title = body.split('|')
    return f'[[{title}|{alias}]]'

print(swap_alias(test_alias))

[[Note Title|i'm aliased text]]


In [17]:
for name, ob in tid_objs.items():
    
    content = ob.content
    
    if not content:
        continue
        
    aliases = alias_ex.findall(content)
    for a in aliases:
        content = content.replace(a, swap_alias(a))

    
    ob.content = content

## Put Pascal words in brackets

Tiddly wiki recognizes words in PascalCase (similar to camelCase) as links, but all Obsidian links must be in double square brackets.

In [18]:
# Regex for matching PascalCase
pascal_ex = re.compile(r'[A-Z][A-Za-z0-9]*[a-z]+[A-Z]+[a-z0-9]*')

for name, ob in tid_objs.items():
    
    content = ob.content
    
    if not content:
        continue
        
    pascals = pascal_ex.findall(content)
        
    # put pascal-cased words in brackets
    for p in pascals:
        linkified = f'[[{p}]]'
        
        if linkified in content: # get out if there's already a link
            continue
            
        if f'[[{p}|' in content: # check for aliases too
            continue
            
        content = content.replace(p, linkified)
        
    ob.content = content

## Write markdown files into the output folder

In [19]:
for t in tid_objs:
    outpath = os.path.join(OUT_FOLDER, t + '.md')
    with open(outpath, 'w') as outfile:
        outfile.write(frontmatter.dumps(tid_objs.get(t)))
print('done')

done
